In [ ]:
import sys,os,os.path
import tensorflow as tf
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,floatX=float32'
from keras import backend as K
sess  = tf.Session()
K.set_session(sess)

In [ ]:
cd 'deep-learning-models/'

In [ ]:
import json
import time
allFiles = 'allImages'
root = '/local_data/dgrossman/dgCars/'
af = open(allFiles,'r')
data = list()
for d in af:
    d = d.strip()
    data.append(json.loads(d))

In [ ]:
from resnet50 import ResNet50
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions
import numpy as np

In [ ]:
s = time.time()
model = ResNet50(weights='imagenet')
print ('loadResNet50',time.time() - s)



In [ ]:
good = list()
bad = list()
file = list()

In [ ]:
interesting = set()

for x in ['police_van','moving_van','car','pickup','suv','truck','crossover','van','minivan','sports_car','cab','racer','convertible','car_wheel','jeep','ambulance']:
    interesting.add(x)

count = 0
im = 0
#with tf.device('/gpu:0'):
#    print ('time:',time.time())
s = time.time()
print ('time:',time.time() - s)
if True:
    for d in data:   
        img_path = '{0}/{1}'.format(root,d['filename'])
        flag = True
        try:
            img = image.load_img(img_path, target_size=(224, 224))

        except:
            #print('FILE :',d['filename'])
            file.append(d)
            flag = False

        if flag:
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            preds = model.predict(x)
            predictions = decode_predictions(preds)[0][:4]
            #out = ''
            found = False
            for prediction in predictions:
                i,t,score = prediction
                #out = d['filename'], prediction
                if t in interesting:
                    #out = 'GOOD'+' ' + d['filename']+' '+t
                    good.append((d,t))
                    found = True
                    break
            if not found:
                bad.append((d,predictions[0][1]))
                #out = 'BAD'+ ' ' + d['filename']+ ' ' + predictions[0][1]
            #print (out)
            #print ('Predicted',decode_predictions(preds)[0][:4],' sec:',time.time() - s )

        atOnce = 10000
        if count == atOnce:
            count = 0
            im = im + 1
            z = time.time() - s
            print('processed:',im * atOnce,'Images','good',len(good),'bad',len(bad),'file',len(file),z)
            s = time.time()
        count = count + 1

In [ ]:
print('processed:',len(good) + len(bad) + len(file),'Images','good',len(good),'bad',len(bad),'file',len(file))

In [ ]:
from collections import defaultdict
q = defaultdict(int)

In [ ]:
for b in good:
    q[b[1]] = q[b[1]]+1

In [ ]:
for qq in q:
    print (qq,q[qq])


In [ ]:
gfile = open('foundCars','w')
    for g in good:
        gfile.write(g+'\n')
gfile.close()

In [ ]:
gfile = open('foundCars','w')
for g in good:
    dat, classification = g
    dat['resnet50'] = classification
    gfile.write(json.dumps(dat)+'\n')
gfile.close()


In [ ]:
good[0]

In [ ]:
7+2